In [1]:
import numpy as np
import pandas as pd
import DataParser

In [2]:
options_dir = "C:/Users/bugin/Desktop/Projects/Variance-Derivatives/20190107/options.csv"
rates_dir = "C:/Users/bugin/Desktop/Projects/Variance-Derivatives/20190107/rates.csv"

options_data = DataParser.GetOptionsData(options_dir)
rates_data = DataParser.GetRateData(rates_dir)
del options_dir, rates_dir

Filter options data to a one month expiration

In [3]:
options_data.EXPIRY = options_data.EXPIRY.apply(lambda x: round(x, 6))
options_mth = options_data[options_data.EXPIRY <= .083333]

In [4]:
import matplotlib.pyplot as plt
import math

Equally weighted options graphs

In [18]:
St = pd.Series(range(2000, 3500)) #set date frame of stock prices
#d1 as input to variance function V
d1 = lambda S,K,sigma,tau: (np.log(S/K)+(sigma**2 * tau)/2)/(sigma * np.sqrt(tau))

In [6]:
import scipy as sp

In [7]:
maxVar = pd.DataFrame(sp.zeros(len(options_data))) #empty dataframe for efficiency

for i in range(maxVar.shape[0]):
    maxVar[i] = max((St * np.sqrt(options_data.EXPIRY[i]) * 
                     np.exp(-d1(St, options_data.STRIKE[i], options_data.IVOL[i], options_data.EXPIRY[i]
                     )**2 /2))/
                    (2 * options_data.EXPIRY[i] * math.sqrt(2 * math.pi)))
#too slow

In [9]:
import numba

In [11]:
@numba.vectorize
def var_calc(S, K, sigma, tau): #s - aray, else scalars
    
    #def d1(S, K, sigma, tau):
    #    value = (np.log(S/K) + (sigma**2 * tau)/2)/(sigma * np.sqrt(tau))
    #    return value
    
    var = np.empty(len(S))
    var = max((S * np.sqrt(tau) * 
                     np.exp(-d1(St, K, sigma, tau
                     )**2 /2))/
                    (2 * tau * math.sqrt(2 * math.pi)))
    return var

In [19]:
#St = St.values
for i in range(maxVar.shape[0]):
    maxVar[i] = var_calc(St.as_matrix, options_data.STRIKE[i], options_data.IVOL[i], options_data.EXPIRY[i])

ValueError: [1mcannot determine Numba type of <class 'method'>[0m

In [ ]:
#weighted
#once we have the unweighted graph append new columb to maxVar that is V_i/K_i^2
#graph as new Y

In [ ]:
maxVar['Var_weighted'] = maxVar[0] / options_data.STRIKE